In [2]:
import pandas as pd
import string

In [13]:
url = "/Users/Artem/Documents/CS 205/NHAHES_VAR_LIST/NHANES Demographics Variable List.html"
df_raw = pd.read_html(url, attrs = {'id' : 'GridView1'})[0]
df_raw.head()

,Variable Name,Variable Description,Data File Name,Data File Description,Begin Year,EndYear,Component,Use Constraints
0,AIALANG,Language of the MEC ACASI Interview Instrument,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,None
1,DMDBORN,In what country {were you/was SP} born?,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,None
2,DMDCITZN,{Are you/Is SP} a citizen of the United States...,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,None
3,DMDEDUC2,(SP Interview Version) What is the highest gra...,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,None
4,DMDEDUC3,(SP Interview Version) What is the highest gra...,DEMO_D,Demographic Variables & Sample Weights,2005,2006,Demographics,None


In [15]:
#exclude restricted vars
df_raw = df_raw[df_raw["Use Constraints"] == 'None']

#drop extra columns
drop_columns = ['Variable Description', 'Data File Description', 'Begin Year', 'EndYear', 'Use Constraints']
df_useful = df_raw.drop(drop_columns, axis = 1)
df_useful.columns = ["feature", "file", 'folder']
df_useful.head()

,feature,file,folder
0,AIALANG,DEMO_D,Demographics
1,DMDBORN,DEMO_D,Demographics
2,DMDCITZN,DEMO_D,Demographics
3,DMDEDUC2,DEMO_D,Demographics
4,DMDEDUC3,DEMO_D,Demographics


In [23]:
#remove year-specific file names
df_useful.file = df_useful.file.apply(func = lambda x: x[:-2] if x[-2] == '_' else x)
df_useful.drop_duplicates(inplace = True)
df_useful.size

510

In [26]:
#rewrite everythig we did above as a function
def parse_file(url):
    df_raw = pd.read_html(url, attrs = {'id' : 'GridView1'})[0]
    df_raw = df_raw[df_raw["Use Constraints"] == 'None']
    drop_columns = ['Variable Description', 'Data File Description', 'Begin Year', 'EndYear', 'Use Constraints']
    df_useful = df_raw.drop(drop_columns, axis = 1)
    df_useful.columns = ["feature", "file", 'folder']
    df_useful.file = df_useful.file.apply(func = lambda x: x[:-2] if x[-2] == '_' else x)
    df_useful.drop_duplicates(inplace = True)
    return df_useful


In [28]:
all_features = pd.DataFrame(columns = ('feature', 'file', 'folder'))

for fname in ["Demographics", "Dietary", "Examination", "Laboratory", "Questionnaire"]:
    url = "/Users/Artem/Documents/CS 205/NHAHES_VAR_LIST/NHANES {} Variable List.html".format(fname)
    print("Parsing", url)
    temp_df = parse_file(url)
    all_features = all_features.merge(temp_df, how = 'outer')
    
print(all_features.shape)
all_features.head()

Parsing /Users/Artem/Documents/CS 205/NHAHES_VAR_LIST/NHANES Demographics Variable List.html
Parsing /Users/Artem/Documents/CS 205/NHAHES_VAR_LIST/NHANES Dietary Variable List.html
Parsing /Users/Artem/Documents/CS 205/NHAHES_VAR_LIST/NHANES Examination Variable List.html
Parsing /Users/Artem/Documents/CS 205/NHAHES_VAR_LIST/NHANES Laboratory Variable List.html
Parsing /Users/Artem/Documents/CS 205/NHAHES_VAR_LIST/NHANES Questionnaire Variable List.html
(17356, 3)


,feature,file,folder
0,AIALANG,DEMO,Demographics
1,DMDBORN,DEMO,Demographics
2,DMDCITZN,DEMO,Demographics
3,DMDEDUC2,DEMO,Demographics
4,DMDEDUC3,DEMO,Demographics


In [29]:
all_features = all_features[all_features.feature != 'SEQN']
print(all_features.shape)
all_features.to_csv("/Users/Artem/Documents/GitHub/CancerPrediction/all_features.csv", index = False)

(16937, 3)


In [ ]:
original_df.to_pickle("./dummy.pkl")